# Fast approximate simulation of seismic waves with deep learning

# Machine learning workshop - student notebook


---

Author: Ben Moseley, Centre for Autonomous Intelligent Machines and Systems, University of Oxford, bmoseley@robots.ox.ac.uk 

This workshop reproduces the results of the paper: *[Fast approximate simulation of seismic waves with deep learning](https://arxiv.org/abs/1807.06873), NeurIPS 2018, B. Moseley, A. Markham and T. Nissen-Meyer*.

Last updated: Jan 2019

---

<img src="figures/header.png" width="600">


## Overview

- **Seismic simulation** is crucial for many geophysical applications, yet traditional approaches are **computationally expensive**.

- In this workshop, we will use **deep learning** to simulate seismic waves.

- We will show that this can offer a **fast approximate alternative** to traditional simulation methods.

---

This workshop takes ~ 1-2 hrs to complete. All the code for this notebook can be found here: https://github.com/benmoseley/seismic-simulation-wavenet

---

## Task

For this proof of principle study, we will consider the simulation of **acoustic waves** propagating in synthetic **horizontally layered** media.

Specifically, we will consider a single fixed point source propagating through a horizonally layered velocity model with 11 fixed receivers horizontally offset from the source, shown below.

<img src="figures/example_simulation.png" width="600"><!---include "" for proper github rendering-->

Left: input velocity model, triangles show receiver locations. Right: wavefield pressure after 1 s, using acoustic Finite-Difference (FD) modelling,  black circle shows fixed point source location.

Our task is as follows:

> Given a randomly selected layered velocity model as input, can we train a neural network to simulate the pressure response recorded at each receiver location?

We wish our neural network to generalise well to velocity models unseen during training. We will compare our results to traditional FD modelling.


## Workflow

We will use the following workflow to complete this task;

- we will preprocess the input velocity profile into its corresponding reflectivity series;

- we will pass this to a deep neural network with a **WaveNet** architecture to simulate the receiver responses;

- we will train this network with many example ground truth FD simulations;

- we will compare the accuracy and computational performance of the trained network to FD simulation.

This workflow is shown below.

<img src="figures/workflow.png" width="850">


## Sections

There are 5 sections to complete in this notebook. This is an interactive notebook; during each section you will be asked to complete some short coding tasks and answer some questions as you go along.

The sections are as follows:

- [Section 1: Data loading and exploration](#Section-1:-Data-loading-and-exploration)

- [Section 2: Designing a deep learning model](#Section-2:-Designing-a-deep-learning-model)

- [Section 3: Training the model](#Section-3:-Training-the-model)

- [Section 4: Evaluating performance](#Section-4:-Evaluating-performance)

- [Section 5: Seismic inversion (optional extra)](#Section-5:-Seismic-inversion-(optional-extra))



## Installation

### 1. Python enviroment

This notebook only requires Python libraries to run. We recommend setting up an new environment, for example
```bash
conda create -n workshop python=3.6  # Use Anaconda package manager
source activate workshop
```
and then installing the following dependencies:
```bash
pip install --ignore-installed --upgrade [packageURL]# install tensorflow (get packageURL from https://www.tensorflow.org/install/pip, see tensorflow website for details)
pip install tqdm requests
conda install matplotlib jupyter
```

Please ensure you install **Tensorflow version 1.10** or above.

Then download the source code:

```bash
git clone https://github.com/benmoseley/seismic-simulation-wavenet.git
```

Finally, check your environment is set up correctly by importing the required libraries for this notebook:

In [ ]:
# import libraries
import os, time
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import main, models, datasets, constants
import io_utils, downloader

### 2. Downloading data

This notebook requires access to a pre-made dataset for training and testing our deep neural network, as well as some pretrained tensorflow model files.

Please use the code below to download this data.

Note: the file size of the seismic dataset is **~600 Mb** and the file size of the pretrained files are **~100 Mb** - ensure you have enough disk space beforehand!

Please **unzip** the pretrained model files after downloading.

In [ ]:
## Download the training data and pretrained model files (~700 Mb total)
io_utils.get_dir("data/")# create data directory
if not os.path.isfile("data/layers_8ms.bin"):# download data
    downloader.get_url(url="https://benmoseley.blog/uploads/DIP/layers_8ms.bin",
                       file_path="data/layers_8ms.bin")
if not (os.path.isfile("data/pretrained.zip") or os.path.isdir("data/pretrained")):# download pretrained model files
    downloader.get_url(url="https://benmoseley.blog/uploads/DIP/pretrained.zip",
                       file_path="data/pretrained.zip")# please unzip this folder after downloading
if os.path.getsize("data/layers_8ms.bin") != 594880000: raise Exception("Try downloading data again, it appears corrupt.")
print("Data downloaded successfully")

# Section 1: Data loading and exploration

In this section we will explore the downloaded dataset. This dataset contains 20,000 example simulations which we will use to train and test our deep neural network.

Each example simulation consists of a horizontally layered velocity model, its corresponding zero-offset reflectivity series and the pressure recorded at each receiver location. The pressure response has been simulated offline using the velocity model and 2D acoustic finite difference modelling. The reflectivity series has also been precomputed offline for you using the velocity model and  a simple 1-D depth to time conversion.

> **Task 1:** use `matplotlib` to visualise some of the examples.

Note:

- the velocity model is sampled in depth with a spacing of 12.5 m;
- the reflectivity model and receiver recordings are sampled in time with a sample rate of 0.008 s;
- The receiver spacing is 200 m.


In [ ]:
### TODO: run the following code to read the provided dataset into Python.
# Note: data/layers_8ms.bin is a flat binary file containing all of the examples.
# The datasets.SeismicDataset class loads examples from the binary file into numpy arrays.
# The constants.Constants class feeds necessary constants to the SeismicDataset class.

# define constants
c = constants.Constants()
c["DATA_PATH"]="data/layers_8ms.bin"# path to dataset
c["N_EXAMPLES"]=20000# number of examples in dataset
c["VELOCITY_SHAPE"]=(236, 1)# (Number of depth steps, 1)
c["REFLECTIVITY_SHAPE"]=(600, 1)# (Number of time steps, 1)
c["GATHER_SHAPE"]=(600, 11)# (Number of time steps, Number of receivers)

# create a SeismicDataset object
dataset = datasets.SeismicDataset(c)
print(len(dataset))

# get the first example from the dataset
velocity_array, reflectivity_array, gather_array = dataset[0]
print(type(velocity_array), velocity_array.shape)
print(type(gather_array), gather_array.shape)

### TODO: Enter you code to visualise a few of the examples here:






> **Question 1:** what formula should be used for converting velocity models to their (zero offset) reflectivity series?

Note: a constant density model was used during simulation. You can check the precomputed reflectivity series are correct by calculating you own reflectivity series and comparing them.


> **Task 2:** plot the histograms of layer velocity values and layer thicknesses across the entire dataset.

Note: the provided velocity models all start at 1500 m/s which leads to a spike in the velocity histogram at this value.

In [ ]:
### TODO: Enter you code here:
# Hint: you can use plt.hist to make the histogram






# Section 2: Designing a deep learning model

We will now define a deep neural network which can simulate the receiver responses, given a velocity model as input.

For a horizontally layered velocity model and horizontally offset receivers, each receiver response is **causally correlated** to the reflectivity series. More precisely, each pressure sample in time is at most correlated to the reflectivity values from previous times. 

We will model the receiver responses using a network which honours this casual relationship. Here, we will choose a **WaveNet** design, shown in the figure below. This architecture was originally designed for speech synthesis (for more info see here: https://deepmind.com/blog/wavenet-generative-model-raw-audio/).

<img src="figures/wavenet.png" width="500">

This model consists of **stacked, casually connected, exponentially dilated convolutional layers**.  The model is causal by design and the exponential dilations allow the field of view of the network to increase exponentially with the depth of the network. 

The input to the WaveNet is the preprocessed reflectivity series corresponding to the input velocity model and the output of the WaveNet is passed to a **final causal convolutional layer with no activation** to generate the output receiver pressure predictions. Each channel in the output tensor corresponds to a receiver prediction.


There are a number of key hyperparameters to decide in this model, including:

- number of hidden layers
- number of hidden channels
- activation function
- filter length of the final convolutional layer

> **Task 3:** define a WaveNet model in Tensorflow and visualise the model's graph using Tensorboard. 

<img src="figures/tensorboard_graph.png" width="800">

Hint: to launch tensorboard run `tensorboard --logdir .` in the current directory, and then navigate your web browser to the address shown.

If you are new to tensorboard, have a read through this guide on visualising graphs using Tensorboard: https://www.tensorflow.org/guide/graph_viz

In [ ]:
### TODO: Run the code below to define a WaveNet model, then visualise the graph using Tensorboard.
### TODO: Vary the hyperparameters of the model to see how this affects the model graph.
### TODO: Look through the source code of wavenet.py and try to understand how the WaveNet is defined.
### the WaveNet source code is also available here: https://github.com/benmoseley/simple-wavenet.

# Note:
# The constants.Constants class is used to hold all model hyperparameters
# The models.SeismicWavenet class is used to define a WaveNet model in tensorflow

tf.reset_default_graph()

# define model hyperparameters
c = constants.Constants()
c["NUM_WAVE_BLOCKS"] = 1# number of WaveNet blocks to use
c["WAVE_HIDDEN_CHANNELS"] = 256# number of hidden channels in WaveNet
c["WAVE_RATES"] = [1,2,4,8,16,32,64,128,256]# dilation rates for each convolutional layer
c["WAVE_BIASES"] = False# whether to use biases in the WaveNet
c["WAVE_ACTIVATION"] = tf.nn.relu# activation function
c["CONV_FILTER_LENGTH"] = 101# filter length of the final output convolutional layer

# define model input and output tensorflow placeholders
velocity = tf.placeholder(shape=(None, 236, 1), dtype=tf.float32, name="velocity")
reflectivity = tf.placeholder(shape=(None, 600, 1), dtype=tf.float32, name="reflectivity")
gather = tf.placeholder(shape=(None, 600, 11), dtype=tf.float32, name="gather")
input_features = {"velocity":velocity, "reflectivity": reflectivity, "gather": gather}

# define model graph
model = models.SeismicWavenet(c, input_features)
model.define_graph()
print(model.y.shape)# the output tensor of the model - check this is the right shape!

# save graph to events file for tensorboard visualisation
io_utils.get_dir("wavenet_graph/")# create data directory
io_utils.clear_dir("wavenet_graph/")# careful - clears all files in this directory
summary_writer = tf.summary.FileWriter("wavenet_graph/",
                                       filename_suffix=".wavenet_graph")# write events file to current directory
with tf.Session() as sess: summary_writer.add_graph(sess.graph)
summary_writer.close()

> **Question 2:** what does the first dimension of the output tensor (model.y) define?

> **Question 3:** how does changing the number of hidden channels affect the network?

> **Question 4:** why do we left-pad each input layer before carrying out a convolution?

> **Question 5:** why is the final output convolutional layer important?

> **Question 6:** what are the total number of trainable parameters in your model?


In [ ]:
### TODO: Write some code to check Question 6 here:
# hint: tf.trainable_variables() lists all trainable variables






# Section 3: Training the model

In this section we will train the WaveNet model we defined in Section 2.

We will use an Adam optimiser with mini-batches of examples, and an $l_{p}$-norm loss function of the form:

$$
L = {1\over N} \lVert G(\hat{Y} - Y)\rVert_{p}^{p}~,
$$

Where $N$ is the mini-batch size, $\hat{Y}$ are the predicted receiver responses, $Y$ are the true receiver responses from FD simulation and $G$ is a gain function of the form $G=t^{2}$, where $t$ is the sample time.

We use this heuristic gain function to increase the weight of later time samples in the pressure responses which are attenuated by spherical divergence and transmission loss.

We will use the first 80% of the data to train the model and withhold the last 20% for testing.

> **Task 4:** define a loss function and optimiser for your model, and visualise the graph using Tensorboard.

In [ ]:
### TODO: Run the code below to define a loss function and optimiser.
### Read through the model.define_loss() source code in models.py to see how
### the loss function and the optimiser operation are defined.
### Visualise the graph using Tensorboard.

# define loss and optimiser hyperparameters
c["BATCH_SIZE"] = 20# number of examples in each min-batch
c["LRATE"] = 1e-5# learning rate for Adam
c["T_GAIN"] = 2# gain exponent
c["L_NUM"] = 2# LP loss number (1 = L1 loss, 2 = L2 loss)

# define loss and optimiser in graph
model.define_loss()

# save graph to events file for tensorboard visualisation
io_utils.get_dir("wavenet_graph_with_loss/")# create data directory
io_utils.clear_dir("wavenet_graph_with_loss/")# careful - clears all files in this directory
summary_writer = tf.summary.FileWriter("wavenet_graph_with_loss/",
                                       filename_suffix=".wavenet_graph_with_loss")# write events file to current directory
with tf.Session() as sess: summary_writer.add_graph(sess.graph)
summary_writer.close()

> **Task 5:** (optional) train your model.

**Please note: training takes ~ 5 hrs on a GPU or >1 day on a CPU. If you do not have the spare time/ computational resource please skip this step and use the pre-trained model file and tensorboard event file provided.**

In [ ]:
### TODO: run this code to train the model.
### YOU CAN SKIP THIS STEP IF YOU DO NOT HAVE THE TIME/COMPUTATIONAL RESOURCE

### This code uses the main.Trainer class to train the model, which periodically saves model
### files and tensorboard summaries to the results/ folder.
### The main.Trainer class also automatically splits the dataset into 80:20 training/test data

### TODO: have a look through the constants.py source code to see some of the other
### constants you can set. For example, when running on a GPU you can 
### set the cuda device with c.DEVICE.

# define some training parameters
c["MODEL_RUN"] = "mytrainedmodel"# name of the training run
c["N_STEPS"] = 100000# number of training steps
c["DEVICE"] = 0# cuda device number
c["SUMMARY_FREQ"] = 1000# how often to report summary statistics

# train the model
tf.reset_default_graph()
run = main.Trainer(c)
#run.train()# uncomment to start training

> **Task 6:** view the training progress in Tensorboard.

If you haven't trained your own model, have a look at the pre-trained model tensorboard event file in the `data/pretrained/forward/` folder provided.

<img src="figures/tensorboard_training.png" width="800">

Note `main.Trainer` also outputs example predictions in the "IMAGES" tab as training progresses.

# Section 4: Evaluating performance

Finally, we will plot some of the simulations from our trained model and compare them to test examples in our dataset.

Use the code below to load one of your saved models from file. If you haven't trained your own model, select the pre-trained model file in `data/pretrained/forward/`.

In [ ]:
### TODO: run the code below to load a saved model from file.

tf.reset_default_graph()

### TODO: IF YOU TRAINED YOUR OWN MODEL: SPECIFY YOUR SAVED MODEL AND ITS CORRESPONDING CONSTANTS FILE HERE
MODEL_LOAD_PATH = "results/models/mytrainedmodel/model.ckpt-XX"
CONSTANTS_LOAD_PATH = "results/summaries/mytrainedmodel/constants_mytrainedmodel.pickle"
###
### OTHERWISE, LOAD THE PRE-TRAINED MODEL BY USING THE FOLLOWING
MODEL_LOAD_PATH = "data/pretrained/forward/model.ckpt-500000"
CONSTANTS_LOAD_PATH = "data/pretrained/forward/constants_forward.pickle"
###

# load the model's saved constants object
c_dict = constants.load_constants_dict(CONSTANTS_LOAD_PATH)
c = constants.Constants(**c_dict)

# define model input and output tensorflow placeholders
velocity = tf.placeholder(shape=(None, 236, 1), dtype=tf.float32, name="velocity")
reflectivity = tf.placeholder(shape=(None, 600, 1), dtype=tf.float32, name="reflectivity")
gather = tf.placeholder(shape=(None, 600, 11), dtype=tf.float32, name="gather")
input_features = {"velocity":velocity, "reflectivity": reflectivity, "gather": gather}

# define and load model
model = models.SeismicWavenet(c, input_features)
model.define_graph()
model.define_loss()
saver = tf.train.Saver()# for loading model
sess = tf.Session()
saver.restore(sess, MODEL_LOAD_PATH)# restore weights

> **Task 7:** Plot the model predictions for some of the unseen velocity models in the test dataset and compare them to their ground truth FD simulations.

> **Task 8:** Measure the time taken to generate 100 predictions.

> **Question 7:** How do these results compare to FD modelling? What are the differences?

Note: each FD simulation typically takes of the order of 1 s to run on a single CPU.

In [ ]:
# Hint: to run some predictions, use the following code:

c["DATA_PATH"]="data/layers_8ms.bin"# ensure data path set correctly in loaded constants file

# grab some test data
dataset = datasets.SeismicDataset(c)
velocity_array, reflectivity_array, gather_array = dataset[16000:16004]# the first four examples in the test dataset
print(reflectivity_array.shape)# (Note slicing a SeismicDataset object returns a batch of examples)

# run a model prediction
gather_prediction_array = sess.run(model.y, feed_dict={reflectivity: reflectivity_array})
print(gather_prediction_array.shape)

### TODO: Enter your code to visualise the model predictions here:





### TODO: Enter your code to time the prediction step here:





### Extension (optional): you could try feeding the model velocity models outside of its training data
### distribution e.g. ones with smoothly varying velocity values (instead of layers) - and evaluate 
### how stable the model predictions appear in this case.

# Section 5: Seismic inversion (optional extra)

We can also use the same WaveNet architecture to carry out **seismic inversion** of this datset: namely, given a set of receiver recordings, can we invert for the underlying velocity model?

For this model, we can simply **reverse the inputs and outputs** of our WaveNet model and retrain the model!

The provided `models.SeismicWavenet` class is expressive enough to handle this; whether the model carries out forward or inverse modelling can be controlled by its `SeismicWavenet(inverse=True)` input flag. When wrapping `SeismicWavenet` with the `main.Trainer` class, this flag can be set via the `constants.Constants` class, ie, setting `c["inverse"]=True`.

> **Optional Task 1**: re-use the code above to train a model which carries out seismic inversion. 

> **Optional Task 2:** Visualise the predictions of your model by adapting your code from Task 7. 

If training is too expensive, you can use the pretrained model file and events file in `data/pretrained/inverse/` provided to help you. 

In [ ]:
tf.reset_default_graph()

# TODO: Enter your code to train/ visualise the inverse modelling here:
# hint: the processing_utils.get_velocity_trace() function can convert a predicted
# reflectivity series to its corresponding velocity trace




